In [13]:
! pip install vaderSentiment textblob

In [55]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from unidecode import unidecode
from textblob import TextBlob
from datetime import datetime, timedelta


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
NFLX = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/NFLX.csv')
FB = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/FB.csv')
AMZN = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMZN.csv')
GOOGL = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/GOOGL.csv')

In [0]:
def data_processing(data, stockcode):
  data = data[['unix_timestamp','content']]
  data = data.dropna()
  data['stockcode'] = data['content'].copy()
  data['stockcode'] = data['stockcode'].apply(lambda x: stockcode)
  data['vader'] = data['content'].copy()
  data['vader'] = data['vader'].apply(lambda x: str(x).lower())
  data['vader'] = data['vader'].apply(lambda x: unidecode(re.sub('[^a-zA-z0-9\s]','',x)))
  data['textblob'] = data['vader'].copy()
  analyzer = SentimentIntensityAnalyzer()
  data['vader'] = data['vader'].apply(analyzer.polarity_scores) 
  data['vader'] = data['vader'].apply(lambda x: x['compound']) 
  data['textblob'] = data['textblob'].apply(TextBlob) 
  data['textblob'] = data['textblob'].apply(lambda x: x.sentiment[0])
  data['unix_timestamp'] = data['unix_timestamp'].apply(lambda x: str(x)[0:10])
  data['date'] = data['unix_timestamp'].apply(lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d'))
  data['unix_timestamp'] = data['unix_timestamp'].apply(lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
  data['unix_timestamp'] = pd.to_datetime(data['unix_timestamp'])
  data['date'] = pd.to_datetime(data['date'])
  data.index = data['date']
  del data['date']
  return data.sort_index() 

In [0]:
NFLX_cleaned = data_processing(NFLX, 'NFLX')
FB_cleaned = data_processing(FB, 'FB')
AMZN_cleaned = data_processing(AMZN, 'AMZN')
GOOGL_cleaned = data_processing(GOOGL, 'GOOGL')

In [0]:
def daily_sentiment_calculator(data):
  data = data[['vader','textblob']].reset_index()
  data = data[['vader','textblob']].groupby(data['date']).sum()
  return data

In [0]:
NFLX_daily = daily_sentiment_calculator(NFLX_cleaned)
FB_daily = daily_sentiment_calculator(FB_cleaned)
AMZN_daily = daily_sentiment_calculator(AMZN_cleaned)
GOOGL_daily = daily_sentiment_calculator(GOOGL_cleaned)

In [0]:
NFLX_cleaned.to_csv('/content/gdrive/My Drive/Colab Notebooks/NFLX_cleaned.csv')
FB_cleaned.to_csv('/content/gdrive/My Drive/Colab Notebooks/FB_cleaned.csv')
AMZN_cleaned.to_csv('/content/gdrive/My Drive/Colab Notebooks/AMZN_cleaned.csv')
GOOGL_cleaned.to_csv('/content/gdrive/My Drive/Colab Notebooks/GOOGL_cleaned.csv')

In [0]:
NFLX_daily.to_csv('/content/gdrive/My Drive/Colab Notebooks/NFLX_daily.csv')
FB_daily.to_csv('/content/gdrive/My Drive/Colab Notebooks/FB_daily.csv')
AMZN_daily.to_csv('/content/gdrive/My Drive/Colab Notebooks/AMZN_daily.csv')
GOOGL_daily.to_csv('/content/gdrive/My Drive/Colab Notebooks/GOOGL_daily.csv')